In [1]:
library(dplyr)
library(tidyr)
library(tibble)
library(lubridate)
library(readr)
library(stringr)
library(ggplot2)
library(data.table)
library(odbc)
library(RMariaDB)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last




#### Connecting to the database



In [2]:
con <- dbConnect(
  drv = RMariaDB::MariaDB(),
  username = "evangelia.paraskevadaki",
  password = "Thaelah8",
  host = "ehr1.deim.urv.cat",
  dbname = "mimiciiiv14",
  port = 3306
)

In [3]:
dbListTables(con)

[1] "ADMISSIONS"         "MICROBIOLOGYEVENTS" "DATETIMEEVENTS"    
 [4] "PROCEDURES_ICD"     "D_ICD_PROCEDURES"   "NOTEEVENTS"        
 [7] "DRGCODES"           "SERVICES"           "PRESCRIPTIONS"     
[10] "D_CPT"              "INPUTEVENTS_CV"     "CALLOUT"           
[13] "CAREGIVERS"         "PROCEDUREEVENTS_MV" "D_ICD_DIAGNOSES"   
[16] "D_LABITEMS"         "DIAGNOSES_ICD"      "LABEVENTS"         
[19] "ICUSTAYS"           "D_ITEMS"            "INPUTEVENTS_MV"    
[22] "CPTEVENTS"          "PATIENTS"           "CHARTEVENTS"       
[25] "OUTPUTEVENTS"       "TRANSFERS"

## Heart Failure cohort



In the practice of medicine one of the factors that you need to always consider is the presence of comorbidities, **diseases that are co-ocurring at the same time than a primary condition**. In this exercise you will explore this concept studying a specific disease, heart failure. As we told you on the introduction of the assignment, this disease has a large prevalence in the U.S. and it&rsquo;s present in the corner&rsquo;s reports of almost 13% of the deaths. Here, you must provide a table with the following information:

-   `ICD9_CODE`: The ICD of the disease
-   `LONG_TITLE`: Long description of the codified disease
-   `N`: Number of patients with that disease associated. You must consider only those diseases that are present on admissions where the primary condition is heart failure (ICD9 starting with 428)
-   `Prevalence`: Prevalence of that disease on the heart failure population (in percentage)

You must order the results by **`Prevalence` in descending order**



In the next step, you will build a **cohort of patients diagnosed with a HF as primary condition**. From each patient, you must obtain some important clinical and demographic features. The resulting table must contain:

-   `SUBJECT_ID`
-   `GENDER`
-   `AGE_FIRST`: age at the first diagnosis of heart failure
-   `AGE_LAST`: age at the last diagnosis of heart failure
-   `ETHNICITY`
-   `DECEASED`: 1 if the patient has died 0 otherwise
-   `AVG_LOS`: Average length-of-stay
-   `DM2`: 1 if the patient has been diagnosed with diabetes mellitus type II, 0 otherwise
-   `CAD`: 1 if the patient has been diagnosed with coronary artery disease, 0 otherwise
-   `CKD`: 1 if the patient has been diagnosed with cronic kidney disease, 0 otherwise
-   `HYPERTENSION`: 1 if the patient has been diagnosed with hypertension, 0 otherwise
    
    The results must be ordered by age **at the last admision in descending order**.



In [4]:
sql1 <- "
SELECT di.SUBJECT_ID, di.ICD9_CODE
FROM DIAGNOSES_ICD di
WHERE di.SEQ_NUM = 1 AND di.ICD9_CODE LIKE '428%'
GROUP BY di.SUBJECT_ID, di.ICD9_CODE;"

hf_primary <- dbGetQuery(con, sql1)

hf_primary %>% head(10)

,SUBJECT_ID,ICD9_CODE
,<int>,<chr>
1,165,42823
2,175,42831
3,176,4280
4,177,4280
5,199,4280
6,205,4280
7,305,42820
8,323,4280
9,351,4280


In [5]:
sql2 <- "
SELECT ce.SUBJECT_ID,
           AVG(CASE WHEN ce.ITEMID IN (211,220045) THEN VALUENUM ELSE NULL END) AS HEARTRATE_avg,
           AVG(CASE WHEN ce.ITEMID IN (615,618) THEN VALUENUM ELSE NULL END) AS RESPRATE_avg, 
           AVG(CASE WHEN ce.ITEMID IN (51,442,455,6701,220179,220050) THEN VALUENUM ELSE NULL END) AS SBP_avg,
           AVG(CASE WHEN ce.ITEMID IN (8368,8440,8441,8555,220180,220051) THEN VALUENUM ELSE NULL END) AS DBP_avg
    FROM CHARTEVENTS ce
    INNER JOIN DIAGNOSES_ICD di ON ce.SUBJECT_ID = di.SUBJECT_ID
    WHERE ce.ITEMID IN (211,220045,615,618,51,442,455,6701,220179,220050,8368,8440,8441,8555,220180,220051)
    AND di.SEQ_NUM = 1 AND di.ICD9_CODE LIKE '428%'
    GROUP BY ce.SUBJECT_ID"

ch_events <- dbGetQuery(con, sql2)

ch_events %>% head(10)

,SUBJECT_ID,HEARTRATE_avg,RESPRATE_avg,SBP_avg,DBP_avg
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,165,66.47826,NA,131.21739,80.56522
2,175,69.70000,16.41918,106.99741,51.41969
3,176,70.25000,16.06250,109.78571,65.00000
4,177,73.74421,20.19306,114.43814,43.35074
5,199,84.39326,20.15556,102.56250,63.53846
6,205,98.79167,20.81944,98.73611,53.37500
7,305,76.75472,16.25653,122.93443,41.22131
8,323,86.64779,19.80000,109.07376,45.51199
9,351,93.05634,19.92647,131.82979,70.82979


In [6]:
sql3 <- "
SELECT di.SUBJECT_ID, p.GENDER, a.ETHNICITY, 
       DATEDIFF(MIN(a.ADMITTIME), p.DOB) / 365 AS AGE_FIRST,
       DATEDIFF(MAX(a.DISCHTIME), p.DOB) / 365 AS AGE_LAST, 
       AVG(DATEDIFF(a.DISCHTIME, a.ADMITTIME)) AS AVG_LOS,
       MAX(a.HOSPITAL_EXPIRE_FLAG) AS DEATH_HOSP,
       MAX(CASE WHEN di.ICD9_CODE LIKE '250%' THEN 1 ELSE 0 END) AS DM2, 
       MAX(CASE WHEN di.ICD9_CODE LIKE '414%' THEN 1 ELSE 0 END) AS CAD,  
       MAX(CASE WHEN di.ICD9_CODE LIKE '585%' THEN 1 ELSE 0 END) AS CKD, 
       MAX(CASE WHEN di.ICD9_CODE LIKE '401%' THEN 1 ELSE 0 END) AS HYPERTENSION,
       MAX(CASE WHEN di.ICD9_CODE IN ('4660', '490', '4910', '4911', '49120', '49121', 
                                      '4918', '4919', '4920', '4928', '494', '4940', 
                                      '4941', '496') THEN 1 ELSE 0 END) AS COPD,
       MAX(CASE WHEN di.ICD9_CODE LIKE '410%' THEN 1 ELSE 0 END) AS AMI,
       MAX(CASE WHEN di.ICD9_CODE IN ('570', '571', '572', '573') THEN 1 ELSE 0 END) AS HepF,
       MAX(icu.ICUSTAY_ID) AS ICU_STAYS,
       AVG(icu.LOS) AS AVG_ICULOS_days
FROM DIAGNOSES_ICD di
INNER JOIN ADMISSIONS a ON di.SUBJECT_ID = a.SUBJECT_ID
INNER JOIN PATIENTS p ON di.SUBJECT_ID = p.SUBJECT_ID
LEFT JOIN ICUSTAYS icu ON di.SUBJECT_ID = icu.SUBJECT_ID
WHERE p.DOB > '2000-01-01'
GROUP BY di.SUBJECT_ID, p.GENDER, a.ETHNICITY, p.DOB
ORDER BY di.SUBJECT_ID; "


tabledata <- dbGetQuery(con, sql3)

tabledata %>% head(30)


,SUBJECT_ID,GENDER,ETHNICITY,AGE_FIRST,AGE_LAST,AVG_LOS,DEATH_HOSP,DM2,CAD,CKD,HYPERTENSION,COPD,AMI,HepF,ICU_STAYS,AVG_ICULOS_days
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int>,<dbl>
1,2,M,ASIAN,0.0000,0.0110,4.0000,0,0,0,0,0,0,0,0,243653,0.09180
2,3,M,WHITE,76.5753,76.6055,11.0000,0,0,0,0,0,0,1,0,211552,6.06460
3,4,F,WHITE,47.8767,47.8959,7.0000,0,0,0,0,0,0,0,0,294638,1.67850
4,5,M,ASIAN,0.0000,0.0055,2.0000,0,0,0,0,0,0,0,0,214757,0.08440
5,6,F,WHITE,65.9836,66.0274,16.0000,0,0,0,0,0,0,0,0,228232,3.67290
6,7,F,WHITE,0.0000,0.0110,4.0000,0,0,0,0,0,0,0,0,278444,0.50340
7,8,M,WHITE,0.0000,0.0110,4.0000,0,0,0,0,0,0,0,0,262299,1.07550
8,9,M,UNKNOWN/NOT SPECIFIED,41.8164,41.8301,5.0000,1,0,0,0,1,0,0,0,220597,5.32310
9,10,F,BLACK/AFRICAN AMERICAN,0.0000,0.0219,8.0000,0,0,0,0,0,0,0,0,288409,8.09210


In [7]:
df <- tabledata %>% inner_join(ch_events, by="SUBJECT_ID")
df %>% head(10)

,SUBJECT_ID,GENDER,ETHNICITY,AGE_FIRST,AGE_LAST,AVG_LOS,DEATH_HOSP,DM2,CAD,CKD,HYPERTENSION,COPD,AMI,HepF,ICU_STAYS,AVG_ICULOS_days,HEARTRATE_avg,RESPRATE_avg,SBP_avg,DBP_avg
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,165,M,BLACK/AFRICAN AMERICAN,86.5397,86.5452,2.00,0,0,1,1,0,1,0,0,247247,0.962800,66.47826,NA,131.21739,80.56522
2,176,M,WHITE,78.9123,78.9178,2.00,0,0,1,0,0,0,0,0,273927,0.514200,70.25000,16.06250,109.78571,65.00000
3,177,M,WHITE,76.6301,77.6959,22.50,0,0,0,0,1,1,1,0,288571,7.601775,73.74421,20.19306,114.43814,43.35074
4,199,M,WHITE,41.7808,48.5507,10.50,0,1,1,0,0,0,1,0,290804,3.586950,84.39326,20.15556,102.56250,63.53846
5,205,F,UNKNOWN/NOT SPECIFIED,75.3342,75.3616,10.00,0,1,1,0,0,0,0,0,277536,2.907600,98.79167,20.81944,98.73611,53.37500
6,305,F,WHITE,72.5507,76.9205,15.50,1,1,1,1,1,0,1,0,297009,3.580720,76.75472,16.25653,122.93443,41.22131
7,323,M,WHITE,52.4274,57.1014,9.75,0,1,1,0,1,0,1,0,282555,5.652450,86.64779,19.80000,109.07376,45.51199
8,351,F,BLACK/AFRICAN AMERICAN,51.1041,51.1288,9.00,0,0,0,0,0,0,0,0,281750,3.859700,93.05634,19.92647,131.82979,70.82979
9,414,M,WHITE,79.6164,79.6438,10.00,1,0,0,1,0,0,0,0,203468,2.653800,82.43284,22.00000,115.30612,48.62245


In [8]:
sql4 <- "
SELECT
  pvt.subject_id, pvt.hadm_id, pvt.icustay_id

  , min(CASE WHEN label = 'ANION GAP' THEN valuenum ELSE NULL END) AS aniongap_min
  , max(CASE WHEN label = 'ANION GAP' THEN valuenum ELSE NULL END) AS aniongap_max
  , min(CASE WHEN label = 'ALBUMIN' THEN valuenum ELSE NULL END) AS albumin_min
  , max(CASE WHEN label = 'ALBUMIN' THEN valuenum ELSE NULL END) AS albumin_max
  , min(CASE WHEN label = 'BANDS' THEN valuenum ELSE NULL END) AS bands_min
  , max(CASE WHEN label = 'BANDS' THEN valuenum ELSE NULL END) AS bands_max
  , min(CASE WHEN label = 'BICARBONATE' THEN valuenum ELSE NULL END) AS bicarbonate_min
  , max(CASE WHEN label = 'BICARBONATE' THEN valuenum ELSE NULL END) AS bicarbonate_max
  , min(CASE WHEN label = 'BILIRUBIN' THEN valuenum ELSE NULL END) AS bilirubin_min
  , max(CASE WHEN label = 'BILIRUBIN' THEN valuenum ELSE NULL END) AS bilirubin_max
  , min(CASE WHEN label = 'CREATININE' THEN valuenum ELSE NULL END) AS creatinine_min
  , max(CASE WHEN label = 'CREATININE' THEN valuenum ELSE NULL END) AS creatinine_max
  , min(CASE WHEN label = 'CHLORIDE' THEN valuenum ELSE NULL END) AS chloride_min
  , max(CASE WHEN label = 'CHLORIDE' THEN valuenum ELSE NULL END) AS chloride_max
  , min(CASE WHEN label = 'GLUCOSE' THEN valuenum ELSE NULL END) AS glucose_min
  , max(CASE WHEN label = 'GLUCOSE' THEN valuenum ELSE NULL END) AS glucose_max
  , min(CASE WHEN label = 'HEMATOCRIT' THEN valuenum ELSE NULL END) AS hematocrit_min
  , max(CASE WHEN label = 'HEMATOCRIT' THEN valuenum ELSE NULL END) AS hematocrit_max
  , min(CASE WHEN label = 'HEMOGLOBIN' THEN valuenum ELSE NULL END) AS hemoglobin_min
  , max(CASE WHEN label = 'HEMOGLOBIN' THEN valuenum ELSE NULL END) AS hemoglobin_max
  , min(CASE WHEN label = 'LACTATE' THEN valuenum ELSE NULL END) AS lactate_min
  , max(CASE WHEN label = 'LACTATE' THEN valuenum ELSE NULL END) AS lactate_max
  , min(CASE WHEN label = 'PLATELET' THEN valuenum ELSE NULL END) AS platelet_min
  , max(CASE WHEN label = 'PLATELET' THEN valuenum ELSE NULL END) AS platelet_max
  , min(CASE WHEN label = 'POTASSIUM' THEN valuenum ELSE NULL END) AS potassium_min
  , max(CASE WHEN label = 'POTASSIUM' THEN valuenum ELSE NULL END) AS potassium_max
  , min(CASE WHEN label = 'PTT' THEN valuenum ELSE NULL END) AS ptt_min
  , max(CASE WHEN label = 'PTT' THEN valuenum ELSE NULL END) AS ptt_max
  , min(CASE WHEN label = 'INR' THEN valuenum ELSE NULL END) AS inr_min
  , max(CASE WHEN label = 'INR' THEN valuenum ELSE NULL END) AS inr_max
  , min(CASE WHEN label = 'PT' THEN valuenum ELSE NULL END) AS pt_min
  , max(CASE WHEN label = 'PT' THEN valuenum ELSE NULL END) AS pt_max
  , min(CASE WHEN label = 'SODIUM' THEN valuenum ELSE NULL END) AS sodium_min
  , max(CASE WHEN label = 'SODIUM' THEN valuenum ELSE NULL END) AS sodium_max
  , min(CASE WHEN label = 'BUN' THEN valuenum ELSE NULL END) AS bun_min
  , max(CASE WHEN label = 'BUN' THEN valuenum ELSE NULL END) AS bun_max
  , min(CASE WHEN label = 'WBC' THEN valuenum ELSE NULL END) AS wbc_min
  , max(CASE WHEN label = 'WBC' THEN valuenum ELSE NULL END) AS wbc_max


FROM
( -- begin query that extracts the data
  SELECT ie.subject_id, ie.hadm_id, ie.icustay_id
  -- here we assign labels to ITEMIDs
  -- this also fuses together multiple ITEMIDs containing the same data
  , CASE
        WHEN itemid = 50868 THEN 'ANION GAP'
        WHEN itemid = 50862 THEN 'ALBUMIN'
        WHEN itemid = 51144 THEN 'BANDS'
        WHEN itemid = 50882 THEN 'BICARBONATE'
        WHEN itemid = 50885 THEN 'BILIRUBIN'
        WHEN itemid = 50912 THEN 'CREATININE'
        WHEN itemid = 50806 THEN 'CHLORIDE'
        WHEN itemid = 50902 THEN 'CHLORIDE'
        WHEN itemid = 50809 THEN 'GLUCOSE'
        WHEN itemid = 50931 THEN 'GLUCOSE'
        WHEN itemid = 50810 THEN 'HEMATOCRIT'
        WHEN itemid = 51221 THEN 'HEMATOCRIT'
        WHEN itemid = 50811 THEN 'HEMOGLOBIN'
        WHEN itemid = 51222 THEN 'HEMOGLOBIN'
        WHEN itemid = 50813 THEN 'LACTATE'
        WHEN itemid = 51265 THEN 'PLATELET'
        WHEN itemid = 50822 THEN 'POTASSIUM'
        WHEN itemid = 50971 THEN 'POTASSIUM'
        WHEN itemid = 51275 THEN 'PTT'
        WHEN itemid = 51237 THEN 'INR'
        WHEN itemid = 51274 THEN 'PT'
        WHEN itemid = 50824 THEN 'SODIUM'
        WHEN itemid = 50983 THEN 'SODIUM'
        WHEN itemid = 51006 THEN 'BUN'
        WHEN itemid = 51300 THEN 'WBC'
        WHEN itemid = 51301 THEN 'WBC'
      ELSE null
    END as label
  , -- add in some sanity checks on the values
  -- the where clause below requires all valuenum to be > 0, so these are only upper limit checks
    CASE
      WHEN itemid = 50862 and valuenum >    10 THEN null -- g/dL 'ALBUMIN'
      WHEN itemid = 50868 and valuenum > 10000 THEN null -- mEq/L 'ANION GAP'
      WHEN itemid = 51144 and valuenum <     0 THEN null -- immature band forms, %
      WHEN itemid = 51144 and valuenum >   100 THEN null -- immature band forms, %
      WHEN itemid = 50882 and valuenum > 10000 THEN null -- mEq/L 'BICARBONATE'
      WHEN itemid = 50885 and valuenum >   150 THEN null -- mg/dL 'BILIRUBIN'
      WHEN itemid = 50806 and valuenum > 10000 THEN null -- mEq/L 'CHLORIDE'
      WHEN itemid = 50902 and valuenum > 10000 THEN null -- mEq/L 'CHLORIDE'
      WHEN itemid = 50912 and valuenum >   150 THEN null -- mg/dL 'CREATININE'
      WHEN itemid = 50809 and valuenum > 10000 THEN null -- mg/dL 'GLUCOSE'
      WHEN itemid = 50931 and valuenum > 10000 THEN null -- mg/dL 'GLUCOSE'
      WHEN itemid = 50810 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 51221 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 50811 and valuenum >    50 THEN null -- g/dL 'HEMOGLOBIN'
      WHEN itemid = 51222 and valuenum >    50 THEN null -- g/dL 'HEMOGLOBIN'
      WHEN itemid = 50813 and valuenum >    50 THEN null -- mmol/L 'LACTATE'
      WHEN itemid = 51265 and valuenum > 10000 THEN null -- K/uL 'PLATELET'
      WHEN itemid = 50822 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 50971 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 51275 and valuenum >   150 THEN null -- sec 'PTT'
      WHEN itemid = 51237 and valuenum >    50 THEN null -- 'INR'
      WHEN itemid = 51274 and valuenum >   150 THEN null -- sec 'PT'
      WHEN itemid = 50824 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 50983 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 51006 and valuenum >   300 THEN null -- 'BUN'
      WHEN itemid = 51300 and valuenum >  1000 THEN null -- 'WBC'
      WHEN itemid = 51301 and valuenum >  1000 THEN null -- 'WBC'
    ELSE le.valuenum
    END as valuenum

  FROM ICUSTAYS ie

  LEFT JOIN LABEVENTS le
    ON le.subject_id = ie.subject_id AND le.hadm_id = ie.hadm_id
    AND le.charttime BETWEEN (DATE_SUB(ie.intime, INTERVAL '6' HOUR)) AND (DATE_ADD(ie.intime, INTERVAL '1' DAY))
    AND le.ITEMID in
    (
      -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
      50868, -- ANION GAP | CHEMISTRY | BLOOD | 769895
      50862, -- ALBUMIN | CHEMISTRY | BLOOD | 146697
      51144, -- BANDS - hematology
      50882, -- BICARBONATE | CHEMISTRY | BLOOD | 780733
      50885, -- BILIRUBIN, TOTAL | CHEMISTRY | BLOOD | 238277
      50912, -- CREATININE | CHEMISTRY | BLOOD | 797476
      50902, -- CHLORIDE | CHEMISTRY | BLOOD | 795568
      50806, -- CHLORIDE, WHOLE BLOOD | BLOOD GAS | BLOOD | 48187
      50931, -- GLUCOSE | CHEMISTRY | BLOOD | 748981
      50809, -- GLUCOSE | BLOOD GAS | BLOOD | 196734
      51221, -- HEMATOCRIT | HEMATOLOGY | BLOOD | 881846
      50810, -- HEMATOCRIT, CALCULATED | BLOOD GAS | BLOOD | 89715
      51222, -- HEMOGLOBIN | HEMATOLOGY | BLOOD | 752523
      50811, -- HEMOGLOBIN | BLOOD GAS | BLOOD | 89712
      50813, -- LACTATE | BLOOD GAS | BLOOD | 187124
      51265, -- PLATELET COUNT | HEMATOLOGY | BLOOD | 778444
      50971, -- POTASSIUM | CHEMISTRY | BLOOD | 845825
      50822, -- POTASSIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 192946
      51275, -- PTT | HEMATOLOGY | BLOOD | 474937
      51237, -- INR(PT) | HEMATOLOGY | BLOOD | 471183
      51274, -- PT | HEMATOLOGY | BLOOD | 469090
      50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
      50824, -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503
      51006, -- UREA NITROGEN | CHEMISTRY | BLOOD | 791925
      51301, -- WHITE BLOOD CELLS | HEMATOLOGY | BLOOD | 753301
      51300  -- WBC COUNT | HEMATOLOGY | BLOOD | 2371
    )
    AND valuenum IS NOT null AND valuenum > 0 -- lab values cannot be 0 and cannot be negative
) pvt
GROUP BY pvt.subject_id, pvt.hadm_id, pvt.icustay_id
ORDER BY pvt.subject_id, pvt.hadm_id, pvt.icustay_id;"


labdata <- dbGetQuery(con, sql4)

labdata %>% head(30)

,subject_id,hadm_id,icustay_id,aniongap_min,aniongap_max,albumin_min,albumin_max,bands_min,bands_max,bicarbonate_min,⋯,inr_min,inr_max,pt_min,pt_max,sodium_min,sodium_max,bun_min,bun_max,wbc_min,wbc_max
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2,163353,243653,NA,NA,NA,NA,1,1,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,0.1,22.0
2,3,145834,211552,15,23,1.8,1.8,2,5,11,⋯,1.3,1.7,13.5,15.7,136,153,41,53,11.3,24.4
3,4,185777,294638,15,15,2.8,2.8,NA,NA,21,⋯,1.1,1.1,12.8,12.8,141,141,10,10,9.7,9.7
4,5,178980,214757,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,13.9,13.9
5,6,107064,228232,20,23,3.0,3.0,NA,NA,15,⋯,1.0,1.4,12.6,14.6,134,138,62,65,10.6,10.6
6,7,118037,236754,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,22.8,22.8
7,7,118037,278444,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
8,8,159514,262299,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,18.7,18.7
9,9,150750,220597,13,13,NA,NA,NA,NA,26,⋯,1.1,1.1,12.7,12.7,136,140,16,17,7.5,13.7


In [24]:

### labdata <- labdata %>% select(-hadm_id, -icustay_id)

labdata_summary <- labdata %>%
  group_by(subject_id) %>%
  summarise(across(everything(), mean, na.rm = TRUE))
labdata_summary

subject_id,aniongap_min,aniongap_max,albumin_min,albumin_max,bands_min,bands_max,bicarbonate_min,bicarbonate_max,bilirubin_min,⋯,inr_min,inr_max,pt_min,pt_max,sodium_min,sodium_max,bun_min,bun_max,wbc_min,wbc_max
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,22.00
3,15,23.0,1.8,1.8,2,5,11.0,25.0,0.80,⋯,1.30,1.7,13.50,15.70,136.0,153.0,41.0,53.0,11.3,24.40
4,15,15.0,2.8,2.8,NaN,NaN,21.0,21.0,1.90,⋯,1.10,1.1,12.80,12.80,141.0,141.0,10.0,10.0,9.7,9.70
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.9,13.90
6,20,23.0,3.0,3.0,NaN,NaN,15.0,18.0,0.20,⋯,1.00,1.4,12.60,14.60,134.0,138.0,62.0,65.0,10.6,10.60
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.8,22.80
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.7,18.70
9,13,13.0,NaN,NaN,NaN,NaN,26.0,30.0,0.40,⋯,1.10,1.1,12.70,12.70,136.0,140.0,16.0,17.0,7.5,13.70
10,17,17.0,NaN,NaN,NaN,NaN,23.0,23.0,4.60,⋯,NaN,NaN,NaN,NaN,133.0,133.0,NaN,NaN,9.2,9.20


In [28]:
labdata_summary <- labdata_summary %>% rename(SUBJECT_ID = subject_id)

In [29]:
dfall <- df %>% inner_join(labdata_summary, by = "SUBJECT_ID")
dfall %>% head(10)

,SUBJECT_ID,GENDER,ETHNICITY,AGE_FIRST,AGE_LAST,AVG_LOS,DEATH_HOSP,DM2,CAD,CKD,⋯,inr_min,inr_max,pt_min,pt_max,sodium_min,sodium_max,bun_min,bun_max,wbc_min,wbc_max
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int64>,<int64>,<int64>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,165,M,BLACK/AFRICAN AMERICAN,86.5397,86.5452,2.00,0,0,1,1,⋯,NaN,NaN,NaN,NaN,140.0000,146.0000,39.00000,41.00000,9.700000,9.700000
2,176,M,WHITE,78.9123,78.9178,2.00,0,0,1,0,⋯,1.70,1.700000,16.50000,16.50000,142.0000,143.0000,22.00000,24.00000,7.600000,8.500000
3,177,M,WHITE,76.6301,77.6959,22.50,0,0,0,0,⋯,1.35,1.375000,14.20000,14.35000,135.2500,140.0000,29.50000,33.75000,12.725000,17.250000
4,199,M,WHITE,41.7808,48.5507,10.50,0,1,1,0,⋯,1.50,1.700000,17.10000,18.50000,138.0000,140.0000,16.00000,17.00000,12.400000,14.050000
5,205,F,UNKNOWN/NOT SPECIFIED,75.3342,75.3616,10.00,0,1,1,0,⋯,2.10,2.500000,17.50000,19.10000,126.0000,128.0000,19.00000,21.00000,8.300000,10.880000
6,305,F,WHITE,72.5507,76.9205,15.50,1,1,1,1,⋯,1.10,1.233333,12.63333,14.03333,137.8000,140.4000,41.80000,43.40000,8.780000,9.840000
7,323,M,WHITE,52.4274,57.1014,9.75,0,1,1,0,⋯,1.20,1.200000,13.30000,13.30000,136.6667,139.3333,70.33333,82.33333,8.333333,9.033333
8,351,F,BLACK/AFRICAN AMERICAN,51.1041,51.1288,9.00,0,0,0,0,⋯,NaN,NaN,NaN,NaN,138.0000,138.0000,5.00000,5.00000,4.600000,4.600000
9,414,M,WHITE,79.6164,79.6438,10.00,1,0,0,1,⋯,1.70,1.800000,17.70000,18.70000,138.0000,140.0000,61.00000,64.00000,5.900000,6.200000


In [30]:
# Save 'labdata' dataframe as a CSV file
write.csv(dfall, "alldata.csv", row.names = FALSE)